In [1]:
import pandas as pd
import numpy as np

In [169]:
marvel_df = pd.read_csv("marvel-wikia-data.csv")
dc_df = pd.read_csv("dc-wikia-data.csv")

In [170]:
marvel_df['Universe'] = 'Marvel'
dc_df['Universe'] = 'DC'

In [171]:
dc_marvel = pd.concat([marvel_df, dc_df], ignore_index=True)
dc_marvel.drop(['GSM', 'YEAR', 'Year', 'urlslug', 'page_id'], axis=1, inplace=True)
dc_marvel

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Universe
0,Spider-Man (Peter Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,Marvel
1,Captain America (Steven Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,Marvel
2,"Wolverine (James \""Logan\"" Howlett)",Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,Marvel
3,"Iron Man (Anthony \""Tony\"" Stark)",Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,Marvel
4,Thor (Thor Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,Marvel
...,...,...,...,...,...,...,...,...,...,...
23267,Nadine West (New Earth),Public Identity,Good Characters,NaN,NaN,Female Characters,Living Characters,NaN,NaN,DC
23268,Warren Harding (New Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,Living Characters,NaN,NaN,DC
23269,William Harrison (New Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,Living Characters,NaN,NaN,DC
23270,William McKinley (New Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,Living Characters,NaN,NaN,DC


In [172]:
dc_marvel.dtypes

name                 object
ID                   object
ALIGN                object
EYE                  object
HAIR                 object
SEX                  object
ALIVE                object
APPEARANCES         float64
FIRST APPEARANCE     object
Universe             object
dtype: object

In [173]:
dc_marvel.dropna(subset='APPEARANCES', inplace=True)

In [174]:
dc_marvel.isna().sum()

name                    0
ID                   5259
ALIGN                3091
EYE                 12447
HAIR                 5822
SEX                   852
ALIVE                   2
APPEARANCES             0
FIRST APPEARANCE      675
Universe                0
dtype: int64

In [175]:
for col in ["ID", "ALIGN", "EYE", "HAIR", "SEX", "FIRST APPEARANCE"]:
    print(col+"\n")
    print(dc_marvel[col].value_counts())
    print("="*50)

ID

Secret Identity                  8207
Public Identity                  6642
No Dual Identity                 1689
Known to Authorities Identity      15
Identity Unknown                    9
Name: ID, dtype: int64
ALIGN

Bad Characters        9076
Good Characters       7007
Neutral Characters    2644
Reformed Criminals       3
Name: ALIGN, dtype: int64
EYE

Blue Eyes             2948
Brown Eyes            2690
Black Eyes             916
Green Eyes             829
Red Eyes               662
White Eyes             471
Yellow Eyes            325
Grey Eyes              128
Hazel Eyes              91
Variable Eyes           49
Purple Eyes             45
Photocellular Eyes      42
Orange Eyes             32
Pink Eyes               23
Gold Eyes               23
Violet Eyes             21
One Eye                 20
Amber Eyes              15
Silver Eyes             11
Multiple Eyes            7
No Eyes                  7
Auburn Hair              7
Yellow Eyeballs          6
Black Eyeballs  

**Category Probability Imputation:** For each missing value, you can calculate the probability distribution of the categories in that column based on the non-missing values and then sample from that distribution to impute the missing value.

In [176]:
def impute_category_prob(df, columns):
    for column in columns:
        probabilities = df[column].value_counts(normalize=True)
        missing_indices = df[df[column].isnull()].index
        df.loc[missing_indices, column] = np.random.choice(probabilities.index, size=len(missing_indices), p=probabilities.values)

np.random.seed(43)
columns_to_impute = ['ID', 'ALIGN', 'HAIR', 'EYE']
impute_category_prob(dc_marvel, columns_to_impute)

In [177]:
dc_marvel.isna().sum()

name                  0
ID                    0
ALIGN                 0
EYE                   0
HAIR                  0
SEX                 852
ALIVE                 2
APPEARANCES           0
FIRST APPEARANCE    675
Universe              0
dtype: int64

In [178]:
for col in ["ID", "ALIGN", "EYE", "HAIR", "SEX", "FIRST APPEARANCE"]:
    print(col+"\n")
    print(dc_marvel[col].value_counts())
    print("="*50)

ID

Secret Identity                  10764
Public Identity                   8774
No Dual Identity                  2248
Known to Authorities Identity       22
Identity Unknown                    13
Name: ID, dtype: int64
ALIGN

Bad Characters        10534
Good Characters        8209
Neutral Characters     3075
Reformed Criminals        3
Name: ALIGN, dtype: int64
EYE

Blue Eyes             6841
Brown Eyes            6369
Black Eyes            2111
Green Eyes            1882
Red Eyes              1504
White Eyes            1096
Yellow Eyes            737
Grey Eyes              295
Hazel Eyes             211
Variable Eyes          117
Purple Eyes            113
Photocellular Eyes     104
Orange Eyes             86
Violet Eyes             55
Gold Eyes               54
Pink Eyes               51
One Eye                 43
Silver Eyes             33
Amber Eyes              31
Auburn Hair             24
Multiple Eyes           21
Yellow Eyeballs         14
No Eyes                 13
Magenta

In [179]:
dc_marvel.loc[dc_marvel['SEX'].isnull(), 'SEX'] = 'Unknown'

In [180]:
dc_marvel.isnull().sum()

name                  0
ID                    0
ALIGN                 0
EYE                   0
HAIR                  0
SEX                   0
ALIVE                 2
APPEARANCES           0
FIRST APPEARANCE    675
Universe              0
dtype: int64

In [181]:
cleaned_df = dc_marvel.dropna()
cleaned_df.head()

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Universe
0,Spider-Man (Peter Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,Marvel
1,Captain America (Steven Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,Marvel
2,"Wolverine (James \""Logan\"" Howlett)",Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,Marvel
3,"Iron Man (Anthony \""Tony\"" Stark)",Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,Marvel
4,Thor (Thor Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,Marvel


In [185]:
cleaned_df.shape

(21144, 10)

In [186]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21144 entries, 0 to 22907
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              21144 non-null  object 
 1   ID                21144 non-null  object 
 2   ALIGN             21144 non-null  object 
 3   EYE               21144 non-null  object 
 4   HAIR              21144 non-null  object 
 5   SEX               21144 non-null  object 
 6   ALIVE             21144 non-null  object 
 7   APPEARANCES       21144 non-null  float64
 8   FIRST APPEARANCE  21144 non-null  object 
 9   Universe          21144 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.8+ MB


In [187]:
cleaned_df.rename(columns={'name': 'Name',
                           'ID': 'Identity', 
                           'ALIGN': 'Alignment', 
                           'EYE': 'Eyes', 
                           'HAIR': 'Hair', 
                           'SEX': 'Sex', 
                           'ALIVE': 'Alive', 
                           'APPEARANCES': 'Appearances',
                           'FIRST APPEARANCE': 'First_appeared'}, inplace=True)
cleaned_df

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1029273803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.rename(columns={'name': 'Name',


,Name,Identity,Alignment,Eyes,Hair,Sex,Alive,Appearances,First_appeared,Universe
0,Spider-Man (Peter Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,Marvel
1,Captain America (Steven Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,Marvel
2,"Wolverine (James \""Logan\"" Howlett)",Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,Marvel
3,"Iron Man (Anthony \""Tony\"" Stark)",Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,Marvel
4,Thor (Thor Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,Marvel
...,...,...,...,...,...,...,...,...,...,...
22903,Gypper Zilich (New Earth),Public Identity,Bad Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,"1941, July",DC
22904,Robert Mason (New Earth),Secret Identity,Neutral Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,"1941, February",DC
22905,Diablo the Horse (New Earth),Public Identity,Good Characters,Red Eyes,Black Hair,Male Characters,Living Characters,1.0,"1940, January",DC
22906,Scott Scanlon (New Earth),Public Identity,Good Characters,Green Eyes,Blond Hair,Male Characters,Living Characters,1.0,"1938, June",DC


In [188]:
cleaned_df['Name'] = cleaned_df['Name'].apply(lambda x: x.split('(')[0])
cleaned_df

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\2491571129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Name'] = cleaned_df['Name'].apply(lambda x: x.split('(')[0])


,Name,Identity,Alignment,Eyes,Hair,Sex,Alive,Appearances,First_appeared,Universe
0,Spider-Man,Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,Marvel
1,Captain America,Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,Marvel
2,Wolverine,Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,Marvel
3,Iron Man,Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,Marvel
4,Thor,No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,Marvel
...,...,...,...,...,...,...,...,...,...,...
22903,Gypper Zilich,Public Identity,Bad Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,"1941, July",DC
22904,Robert Mason,Secret Identity,Neutral Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,"1941, February",DC
22905,Diablo the Horse,Public Identity,Good Characters,Red Eyes,Black Hair,Male Characters,Living Characters,1.0,"1940, January",DC
22906,Scott Scanlon,Public Identity,Good Characters,Green Eyes,Blond Hair,Male Characters,Living Characters,1.0,"1938, June",DC


In [189]:
cleaned_df['Identity'].replace({'Secret Identity': 'Secret',
                                'Public Identity': 'Public',
                                'No Dual Identity': 'Non-dual',
                                'Known to Authorities Identity': 'Known to Authorities',
                                'Identity Unknown': 'Unknown'}, inplace=True)

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\3815805683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Identity'].replace({'Secret Identity': 'Secret',


In [190]:
cleaned_df.Identity.value_counts()

Secret                  10452
Public                   8585
Non-dual                 2072
Known to Authorities       22
Unknown                    13
Name: Identity, dtype: int64

In [191]:
cleaned_df['Alignment'].replace({'Good Characters': 'Good',
                                 'Neutral Characters': 'Neutral',
                                 'Bad Characters': 'Bad',
                                 'Reformed Criminals': 'Bad'}, inplace=True)

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\2349068688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Alignment'].replace({'Good Characters': 'Good',


In [192]:
cleaned_df.Alignment.value_counts()

Bad        10250
Good        7957
Neutral     2937
Name: Alignment, dtype: int64

In [193]:
cleaned_df['Eyes'] = cleaned_df['Eyes'].apply(lambda x: x.split()[0])

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\3431376347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Eyes'] = cleaned_df['Eyes'].apply(lambda x: x.split()[0])


In [194]:
cleaned_df.Eyes.value_counts()

Blue             6656
Brown            6170
Black            2054
Green            1819
Red              1459
White            1051
Yellow            720
Grey              286
Hazel             202
Variable          113
Purple            107
Photocellular     104
Orange             84
Violet             53
Gold               51
Pink               48
One                41
Silver             32
Amber              27
Auburn             24
Multiple           19
No                 13
Magenta             7
Compound            4
Name: Eyes, dtype: int64

In [195]:
cleaned_df['Hair'].replace({'Strawberry Blond Hair': 'Red Hair',
                            'Light Brown Hair': 'Brown Hair',
                            'Reddish Blond Hair': 'Auburn Hair',
                            'Platinum Blond Hair': 'Silver Hair',
                            'Orange-brown Hair': 'Orange Hair', 
                            'Dyed Hair': 'Blond Hair',
                            'Reddish Brown Hair': 'Red Hair',
                            'Bronze Hair': 'Blond Hair'}, inplace=True)

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1459450016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Hair'].replace({'Strawberry Blond Hair': 'Red Hair',


In [196]:
cleaned_df['Hair'] = cleaned_df['Hair'].apply(lambda x: x.split()[0])

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\474703024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Hair'] = cleaned_df['Hair'].apply(lambda x: x.split()[0])


In [197]:
cleaned_df.Hair.value_counts()

Black       6688
Brown       4552
Blond       2927
Red         1481
White       1398
No          1387
Bald        1067
Grey         877
Green        193
Blue         117
Auburn       110
Purple        96
Orange        88
Pink          43
Variable      38
Yellow        28
Silver        24
Gold          15
Magenta        9
Violet         6
Name: Hair, dtype: int64

In [198]:
cleaned_df['Sex'] = cleaned_df['Sex'].apply(lambda x: x.split()[0])

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1672294193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Sex'] = cleaned_df['Sex'].apply(lambda x: x.split()[0])


In [199]:
cleaned_df.Sex.replace({'Agender': 'Genderless'}, inplace=True)

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1302069223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.Sex.replace({'Agender': 'Genderless'}, inplace=True)


In [200]:
cleaned_df.Sex.value_counts()

Male           14964
Female          5301
Unknown          824
Genderless        52
Genderfluid        2
Transgender        1
Name: Sex, dtype: int64

In [201]:
cleaned_df.Alive = cleaned_df.Alive.map({'Living Characters': 'Yes',
                                          'Deceased Characters': 'No'})

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\606647035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.Alive = cleaned_df.Alive.map({'Living Characters': 'Yes',


In [202]:
cleaned_df.Alive.value_counts()

Yes    16050
No      5094
Name: Alive, dtype: int64

In [203]:
cleaned_df.Appearances = cleaned_df.Appearances.astype('int64')

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1654084432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.Appearances = cleaned_df.Appearances.astype('int64')


In [204]:
cleaned_df['First_appeared'].sort_values()[0:20]

16607      1935, October
22907     1936, February
18084     1936, February
16697     1936, February
17844     1936, February
18312     1936, February
17081     1936, February
16943        1936, March
19216    1936, September
18605        1937, March
16761        1937, March
16492        1937, March
16557        1937, March
17374      1938, January
17117         1938, July
17406         1938, June
17438         1938, June
16722         1938, June
16458         1938, June
16390         1938, June
Name: First_appeared, dtype: object

In [205]:
d={'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April',
    'May': 'May', 'Jun': 'June', 'Jul': 'July', 'Aug': 'August', 
    'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}

def extract_year(x):
    y = x.split('-')
    z = x.split(',')
#     m=''
    if len(y) == 2:
        x=y[1]
        m=d[y[0]]
        
        if int(x) < 15:
            x = "20"+x
        else:
            x="19"+x
        return x+', '+m
#     if len(z) == 2:
#         x=z[0]
    else:
        return x
    

In [206]:
cleaned_df.First_appeared = cleaned_df['First_appeared'].apply(extract_year)

C:\Users\relat\AppData\Local\Temp\ipykernel_34340\1454266243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.First_appeared = cleaned_df['First_appeared'].apply(extract_year)


In [207]:
cleaned_df

,Name,Identity,Alignment,Eyes,Hair,Sex,Alive,Appearances,First_appeared,Universe
0,Spider-Man,Secret,Good,Hazel,Brown,Male,Yes,4043,"1962, August",Marvel
1,Captain America,Public,Good,Blue,White,Male,Yes,3360,"1941, March",Marvel
2,Wolverine,Public,Neutral,Blue,Black,Male,Yes,3061,"1974, October",Marvel
3,Iron Man,Public,Good,Blue,Black,Male,Yes,2961,"1963, March",Marvel
4,Thor,Non-dual,Good,Blue,Blond,Male,Yes,2258,"1950, November",Marvel
...,...,...,...,...,...,...,...,...,...,...
22903,Gypper Zilich,Public,Bad,Black,Black,Male,Yes,1,"1941, July",DC
22904,Robert Mason,Secret,Neutral,Black,Black,Male,Yes,1,"1941, February",DC
22905,Diablo the Horse,Public,Good,Red,Black,Male,Yes,1,"1940, January",DC
22906,Scott Scanlon,Public,Good,Green,Blond,Male,Yes,1,"1938, June",DC


In [221]:
final_df = cleaned_df.sample(frac=1).reset_index(drop=True)
final_df

,Name,Identity,Alignment,Eyes,Hair,Sex,Alive,Appearances,First_appeared,Universe
0,Ikonn,Secret,Neutral,Blue,No,Male,Yes,6,"1981, June",Marvel
1,Carol Ferris,Public,Neutral,Blue,Black,Female,Yes,319,"1959, October",DC
2,Vincent Edge,Public,Bad,Brown,White,Male,Yes,16,"1989, September",DC
3,Thomas Adkins,Public,Bad,Brown,Black,Male,Yes,1,"2002, April",Marvel
4,Razorback,Secret,Bad,Blue,Black,Male,No,1,"1988, February",DC
...,...,...,...,...,...,...,...,...,...,...
21139,Mithra,Public,Good,Yellow,Brown,Male,Yes,1,"1974, October",Marvel
21140,Gayle Cord-Becker,Secret,Good,Blue,Brown,Female,No,1,"1994, September",Marvel
21141,Crazy Legs,Secret,Bad,Blue,Brown,Male,Yes,4,"1987, August",Marvel
21142,Rio Ferdinand,Secret,Bad,Blue,Black,Female,Yes,8,"2006, October",DC


In [222]:
final_df.to_csv('comic_characters.csv')